In [22]:
import os
from pydrive.drive import GoogleDrive 

def delete_items_in_gdrive_folder(gauth, folder_id):
	
	""" Passed in authentication to drive object """
	drive = GoogleDrive(gauth)
	print(drive)

	""" Iterate through passed passed gdrive folder (via folder_id) and delete all the items in it."""
	item_list = drive.ListFile({'supportsAllDrives':True, 'includeItemsFromAllDrives':True, 'q': "'{folder_id}' in parents and trashed=false".format(folder_id=folder_id)}).GetList()
	for item in item_list:
		print('title: %s, id: %s' % (item['title'], item['id']))
		delete_item = drive.CreateFile({'supportsAllDrives':True, 'id': item['id']})
		delete_item.Trash({'supportsAllDrives':True})
		#print('Deleted title: %s, id: %s' % (item['title'], item['id']))

	print('delete done!')

def upload_handler(gauth, os_root_path, gdrive_root_folder_id):
	# Google authentication 
	drive = GoogleDrive(gauth)
	dir_dict = {os_root_path: gdrive_root_folder_id}

	#For each dir path starting from os_root_path by os.walk()
	for dirpath, dirnames, files in os.walk(os_root_path):
		print(dirpath, dirnames)
		#print(files)
		for file in files:
			#print(dir_dict, file, str('in ' + dirpath))
			new_file = drive.CreateFile({'supportsAllDrives':True ,'title': file, "parents": [{"kind": "drive#fileLink", "id": dir_dict[dirpath]}]})  # Create GoogleDriveFile instance to parentID.
			file_path = os.path.join(dirpath, file)
			new_file.SetContentFile(file_path) # Set content of the file from given string.
			print(new_file)
			new_file.Upload({'supportsAllDrives':True})
		#for this path create a folder
		for dirname in dirnames:
			#print(dirname)
			new_folder = drive.CreateFile({'supportsAllDrives': True, 'title': dirname, "parents":  [{"id": dir_dict[dirpath]}], "mimeType": "application/vnd.google-apps.folder" })
			new_folder.Upload({'supportsAllDrives':True})
			dir_dict[os.path.join(dirpath, dirname)] = new_folder['id']
			#print(new_folder['id'])
	
	#print(dir_dict)	

In [23]:
from pydrive.auth import GoogleAuth
from datetime import datetime

def upload(path, folder_id):

	""" Initial Authentication, local """
	gauth = GoogleAuth()
	gauth.LocalWebserverAuth()

	# Gdrive folder id, if you want hard code it
	#folder_id = ''
	
	#print(drive, vars(gauth))

	# Empty Gdrive folder
	delete_items_in_gdrive_folder(gauth, folder_id)

	#Path to folder for sync if you want to hard code it
	#path = '' 

	#Upload handler for folder path and Grdrive folder
	upload_handler(gauth, path, folder_id)

	with open('./update_log.txt', 'a') as update_log:
		update_log.write( '\n' + 'Synced with GDrive on ' + str(datetime.now()))


""" If you decide to hard code the Google Drive folder ID and Directory Path into the upload function 
	you dont have to read the folder_sync_register below. You can just remove the code and uncomment
	the upload call below this section
"""

import csv
with open('folder_sync_register.txt', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
    	upload(row[0], row[1])
    
#upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=248540739780-pgq7icu3dkknj2m51aloopqo5ept3vut.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
delete done!
E:\GitHub\WF-TDM-v9.0\Scenarios ['2023-02-23-MCCalib', '_default']
E:\GitHub\WF-TDM-v9.0\Scenarios\2023-02-23-MCCalib ['BY_2019']
E:\GitHub\WF-TDM-v9.0\Scenarios\2023-02-23-MCCalib\BY_2019 ['0_InputProcessing', '1_HHDisag_AutoOwn', '2_TripGen', '3_Distribute', '4_ModeChoice', '5_AssignHwy', '6_REMM', '7_PostProcessing', 'Temp', '_Log']
GoogleDriveFile({'supportsAllDrives': True, 'title': '0GeneralParameters.block', 'parents': [{'kind': 'drive#fileLink', 'id': '1C8F9c6y3kVcgOeQHOdn9VACypoiap5C8'}], 'mimeType': None})
GoogleDriveFile({'supportsAllDrives': True, 'title': '11_12_Mc_HBW_HBO_NHB_HBC.txt', 'parents': [{'kind': 'drive#fileLink', 'id':

KeyboardInterrupt: 